# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from collections import namedtuple, deque

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
!wandb login xyz

Successfully logged in to Weights & Biases!


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Adam/.netrc


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
env = UnityEnvironment(file_name='./reacher/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


## Init AI

New run ...

Hyper parameters...

In [53]:
import wandb

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 512        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-3         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0.0   # L2 weight decay
OU_SIGMA = 0.20
OU_THETA = 0.15
LEARN_INTERVAL = 6
LEARN_NUM = 2

config = {
    "BUFFER_SIZE": BUFFER_SIZE,
    "BATCH_SIZE": BATCH_SIZE,
    "GAMMA": GAMMA,
    "TAU": TAU,
    "LR_ACTOR": LR_ACTOR,
    "LR_CRITIC": LR_CRITIC,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "OU_SIGMA": OU_SIGMA,
    "OU_THETA" : OU_THETA,
    "LEARN_INTERVAL": LEARN_INTERVAL,
    "LEARN_NUM": LEARN_NUM
}
wandb.init(config=config, project="reacher_ddpg_continuous_control")

INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOnJ6MGY1aDN6OnJlYWNoZXJfZGRwZ19jb250aW51b3VzX2NvbnRyb2w6YWRhbV9ibHZjaw==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\config.yaml
INFO:wandb.run_manager:saving patches
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


W&B Run: https://app.wandb.ai/adam_blvck/reacher_ddpg_continuous_control/runs/rz0f5h3z

INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:file/dir created: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json
INFO:wandb.run_manager:file/dir created: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:file/dir created: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir created: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl
INFO:wandb.run_manager:file/dir created: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuou

Device...

In [54]:
import torch
from ddpg_agent import Agent

In [55]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Create AI agent...

In [57]:
agent = Agent( device, state_size, num_agents, action_size, 1, \
                BUFFER_SIZE, BATCH_SIZE, GAMMA, TAU, LR_ACTOR, LR_CRITIC, WEIGHT_DECAY, \
                 LEARN_INTERVAL, LEARN_NUM, OU_SIGMA, OU_THETA)

wandb.watch(agent.actor_local)
wandb.watch(agent.critic_local)

## Train AI

In [58]:
def save_agent(agent, checkpoint_name):
    torch.save(agent.actor_local.state_dict(), checkpoint_name+'_actor.pth')
    torch.save(agent.critic_local.state_dict(), checkpoint_name+'_critic.pth')

def load_agent(agent, checkpoint_name):
    agent.actor_local.load_state_dict(torch.load(checkpoint_name+'_actor.pth'))
    agent.critic_local.load_state_dict(torch.load(checkpoint_name+'_critic.pth'))
    
def train_ddpg(n_episodes=2000, max_t=1000, checkpoint_name='checkpoint'):
    scores = []
    scores_window = deque(maxlen=100)
    max_score = np.Inf
    
    for i_episode in range(1, n_episodes+1):
        
        # reset => env, agent, score
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        agent.reset()
        score = np.zeros(num_agents)
        
        # run for maxt_ time (or when `done` is reached)
        # - Agent decides action - agent.act
        # - Environement accepts action - env.step
        # - Agent is informed about taken action - agent.step
        for t in range(max_t):
            
            # Mr AI, what action should we take?
            actions = agent.act(states)
            
            # Take the action in the environment
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            # Mr AI, if you choose to accept (for every human instance)
            agent.step(t, states, actions, rewards, next_states, dones)
            
            # Loop around & keep track of score
            states = next_states
            score += rewards
            
            if np.any(dones): # if any agent is at it's completion
                break
                
        # log
        scores_window.append(np.mean(score))
        scores.append(np.mean(score))
        
        # log avg score for this episode across agents
        wandb.log({"mean_score": np.mean(score)})
        avg_score = np.mean(scores_window)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, \
                                                                          np.mean(scores_window), \
                                                                          np.mean(score) ,\
                                                                          end=""))
        
        # log avg score in last 100 episodes
        if i_episode % 20 == 0 or avg_score > 30.0:
            save_agent(agent, checkpoint_name)
            wandb.log({"window_score": np.mean(scores_window)})
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))   
        
        if avg_score >= 31.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage score: {:.2f}'.format(i_episode, average_score))
            break
        
    return scores

In [59]:
scores = train_ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOnJ6MGY1aDN6OnJlYWNoZXJfZGRwZ19jb250aW51b3VzX2NvbnRyb2w6YWRhbV9ibHZjaw==
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving patches
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-lear

Episode 1	Average Score: 0.83	Score: 0.83


INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json


Episode 2	Average Score: 0.78	Score: 0.72


INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json


...

Episode 115	Average Score: 30.77	Score: 36.88
Episode 115	Average Score: 30.77


INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json


Episode 116	Average Score: 31.12	Score: 37.21
Episode 116	Average Score: 31.12


NameError: name 'average_score' is not defined

INFO:wandb.run_manager:shutting down system stats and metadata service
INFO:wandb.run_manager:stopping streaming files and file change observer
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-summary.json
INFO:wandb.run_manager:file/dir modified: D:\Code\DRL_udacity\deep-reinforcement-learning\p2_continuous-control\wandb\run-20191120_144308-rz0f5h3z\wandb-events.jsonl


## Observe AI

In [ ]:
load_agent(agent, 'checkpoint')

In [61]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
score = np.zeros(num_agents)                          # initialize the score (for each agent)

while True:

    # Mr AI, what action should we take?
    actions = agent.act(states)

    # Take the action in the environment
    env_info = env.step(actions)[brain_name]
    next_states = env_info.vector_observations
    rewards = env_info.rewards
    dones = env_info.local_done

    # Mr AI, here are the results of your action, and the next state
    # agent.step(states, actions, rewards, next_states, dones)

    # Loop around & keep track of score
    states = next_states
    score += rewards

    if np.any(dones): # if any agent is at it's completion
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(score)))

Total score (averaged over agents) this episode: 37.15949916942045


In [ ]:
env.close()